<a href="https://colab.research.google.com/github/ProfEddie/HypCLIP/blob/perceiver/lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from model.modules.dct import dct
from lavis.datasets.builders import load_dataset
from utils.data_utils import  get_loaders
from lavis.models import load_model_and_preprocess
import torch
import torch.nn as nn
import torch.nn.functional as  F
import matplotlib.pyplot as plt


/home/jarvis/anaconda3/envs/HypCLIP/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-01-10 02:30:44,889] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
COCO_PATH = "/mnt/data/itr_dataset/dataset/coco/images"
FLICKR_PATH = "/mnt/data/itr_dataset/dataset/flickr30k/flickr30k_images"
dataset = load_dataset("flickr30k", vis_path=FLICKR_PATH, cfg_path=None)

Using downloaded and verified file: /mnt/data/itr_dataset/dataset/flickr30k/annotations/train.json
Using downloaded and verified file: /mnt/data/itr_dataset/dataset/flickr30k/annotations/val.json
Using downloaded and verified file: /mnt/data/itr_dataset/dataset/flickr30k/annotations/test.json


In [3]:

def plot_hidden_states(hidden_states, filtered=False):
  for hidden_state in hidden_states:
    hidden_state=hidden_state.permute(1,0,2)
    x_dct = dct(hidden_state.transpose(0,2), norm='ortho').transpose(0,2)
    if not filtered:
      numpy_array = ((x_dct.permute(1,0,2).mean(0).mean(1))).cpu().detach().numpy()
    else:
      numpy_array = ((x_dct.permute(1,0,2).mean(0).mean(1)))[:350].cpu().detach().numpy()
    plt.figure(figsize=(10, 2))

    # plt.imshow(numpy_array, cmap='viridis')  # You can choose a different colormap
    plt.plot(numpy_array)# You can choose a different colormap
    # plt.colorbar()
    plt.show()



In [4]:
from config import parser, config_dict, config_args, add_flags_from_config, argparse

parser = argparse.ArgumentParser()
for _, config_dict in config_args.items():
    parser = add_flags_from_config(parser, config_dict)


In [5]:
from lavis.models import load_model_and_preprocess
from model.modules.compressed_models import CompressedLAVISBLIP, CompressedLAVISBLIP2


model, vis_processors, txt_processors = load_model_and_preprocess("blip_retrieval", "flickr", is_eval=False)
train_loader, val_loader, test_loader = get_loaders(
    10, 
    dataset,
    vis_processor=vis_processors['eval'],
    txt_processor=txt_processors['eval'],
    tokenizer=model.tokenizer,
)
device = torch.device('cpu')
model = CompressedLAVISBLIP(model, compress_method='dct')


In [6]:
model

CompressedLAVISBLIP(
  (vision_model): VisionTransformerEncoder(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=Fal

In [9]:
from transformers import AutoModel, AutoProcessor
from model.modules.compressed_models import CompressedHFCLIP
BLIP_BASE_COCO = "Salesforce/blip-itm-base-coco"
BLIP_LARGE_COCO = "Salesforce/blip-itm-large-coco"
BLIP_2 = "Salesforce/blip2-opt-2.7b"
CLIP_BASE_PATCH_32 = "openai/clip-vit-base-patch32"
CLIP_BASE_PATCH_16 = "openai/clip-vit-base-patch16"
CLIP_LARGE_PATCH_14 = "openai/clip-vit-large-patch14"
BLIP_BASE = "Salesforce/blip-image-captioning-base"
BLIP_BASE_FLICKR = "Salesforce/blip-itm-base-flickr"
BLIP_LARGE_FLICKR = "Salesforce/blip-itm-large-flickr"
BLIP_LARGE_FLICKR = "Salesforce/blip-itm-large-coco"
LAVIS_BLIP_BASE_FLICKR = "lavis-blip-itm-base-flickr"
LAVIS_BLIP_BASE_COCO= "lavis-blip-itm-base-coco"
model_ckt = CLIP_BASE_PATCH_32 
ori_model = AutoModel.from_pretrained(model_ckt)
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
# model = CompressedHFCLIP(ori_model, 'dct').to(device)
processor = AutoProcessor.from_pretrained(model_ckt)
train_loader, val_loader, test_loader = get_loaders(
    40, 
    dataset,
    vis_processor=processor,
    txt_processor=None,
    tokenizer=processor,
)

In [ ]:
for batch in train_loader:
    print(batch['pixel_values'].shape)
    break

In [ ]:
clip_output = ori_model.vision_model(
    pixel_values=batch['pixel_values'], 
    output_hidden_states=True,
    output_attentions=True,
    return_dict=True
)

In [11]:
import pandas as pd
import numpy as np
total_coef = [0]*len(clip_output.attentions)
for i in range(len(clip_output.attentions)):
    df = pd.DataFrame()
    attention = clip_output.attentions[i][:,:,:,1:].mean(0).mean(0).mean(0)
    attention -= attention.min() 
    attention /= attention.max() 
    feature_std = clip_output.hidden_states[i][:,1:,:].mean(0).std(1)
    feature_std -= feature_std.min() 
    feature_std /= feature_std.max() 
    df['attn'] = attention.detach().numpy()
    df['feat_std'] = feature_std.detach().numpy()
    plt.figure(figsize=(10, 5))
    plt.plot( df['feat_std'] )
    plt.plot( df['attn'] )
    
    correlation_matrix = df[['attn', 'feat_std']].corr()
    correlation_coefficient = correlation_matrix.loc['attn', 'feat_std']
    total_coef[i] += correlation_coefficient
    
plt.figure(figsize=(10, 5))
plt.plot(np.array(total_coef))
    
    
    

NameError: name 'clip_output' is not defined

In [12]:
import numpy as np
import matplotlib.pyplot as plt

def plot_coeff_attn_feat_std(coeff):
# Generate sample data


    # Set the threshold value
    threshold = 0.5 
    x = np.linspace(0, len(coeff), len(coeff))
    y = np.array(coeff)

    # Create two masks for values above and below the threshold
    mask_above = x >= len(coeff)/2 
    mask_below = y < len(coeff)/2 

    # Plot values above the threshold in one color
    plt.plot(x, y, color='blue', linestyle='-', label='Connected Line')
    plt.plot(x[mask_above], y[mask_above], color='orange', label='Decision making')

    # Plot values below the threshold in another color
    # plt.plot(x[mask_below], y[mask_below], color='red', label='Below Threshold')
    
    # Add a horizontal line to indicate the threshold
    plt.axhline(y=threshold, color='black', linestyle='--', label='Feature extract')
    plt.grid(True, linestyle='--', alpha=0.7)

    # Add labels and title
    plt.xlabel('Layers')
    plt.ylabel('Correlation coefficient')
    plt.title('Correlation coefficient between attention scores and std of features in each vector')

    # Add legend
    plt.legend()

    # Show the plot
    plt.show()

plot_coeff_attn_feat_std(total_coef)

NameError: name 'total_coef' is not defined